### How to make api 

In [1]:
from wildcatpy.api_calls import * 

In [2]:
!python3 -m pip install python-dotenv


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

username = os.getenv("USERNAME") # you can also type your password here manually
password = os.getenv('PASSWORD') # You can also type your username here manually


In [4]:
api_call = WildcatApi(username,password)

In [5]:
# with this private func we can make a call so we can test new api's 

add = "/map/all/describe" # this is what has to be added to standard ->https://focus.sensingclues.org/api/
output = api_call._api_call("get", add, {}).json()

In [6]:
# this is the json file I made to extract the correct values 
# What it does it it goes deeper in the dict (models) and starts looping if possible
# Per row it will extract the values in the extract_values argument
# It will explode every row with values that are in the explode_values
# What happens now it makes a new row for the default and for the tracks
# But both are containing the extract_values 
# We fix this afterwards by filtering. Normally we don't want seperated rows when exploding muliple times
{
  "cols_to_data": [
    "models"
  ],
  "extractor": {
    "default": {
      "extract_values": [
        "name" # it will extract the name for values in default 
      ],
      "layers": { 
        "explode_values": [ # means it will add a new row for every of those columns in the default
          "id",
          "description",
          "geometryType"
        ]
      }
    },
    "track": {
      "extract_values": [
        "name",  # it will extract the name, id for values in track 
        "id"
      ],
      "layers": { 
        "explode_values": [ # means it will add a new row for every of those columns in the track
          "id",
          "description",
          "geometryType"
        ]
      }
    }
  }
}

{'cols_to_data': ['models'],
 'extractor': {'default': {'extract_values': ['name'],
   'layers': {'explode_values': ['id', 'description', 'geometryType']}},
  'track': {'extract_values': ['name', 'id'],
   'layers': {'explode_values': ['id', 'description', 'geometryType']}}}}

In [7]:
extr = dataExtractor("all_layers")
extr.extr(output, nested_col_names=True)

[{'default_name': 'default',
  'track_name': 'track',
  'track_id': 'track',
  'default_layers_id': 0,
  'default_layers_description': 'All Point geometries for Observations',
  'default_layers_geometryType': 'Point'},
 {'default_name': 'default',
  'track_name': 'track',
  'track_id': 'track',
  'default_layers_id': 1,
  'default_layers_description': 'All Point geometries for Track',
  'default_layers_geometryType': 'Point'},
 {'default_name': 'default',
  'track_name': 'track',
  'track_id': 'track',
  'default_layers_id': 2,
  'default_layers_description': 'All Point geometries for Agent',
  'default_layers_geometryType': 'Point'},
 {'default_name': 'default',
  'track_name': 'track',
  'track_id': 'track',
  'track_layers_id': 0,
  'track_layers_description': 'All MultiPoint geometries for Track',
  'track_layers_geometryType': 'MultiPoint'}]

In [ ]:
# this is a very simple one so we just make a standard private function in the class
# This should be a private function because it is only used by other functions and not by the users
# Private means that you start the function with a _  (also not visible in documenation by user then)

# To test this new function we extend the WildcatAPi Class 
# Then we make the variable url_addition
# Then use the _api_call function and provide the type request, url_addition and payload if needed
# This function returns all the models
# Normally we use an extraction but since this function returns almost everything it is not needed.
# If the function works here you can implement it in the api_calls.py

class testNewAPi(WildcatApi):
    def _get_all_layers(self):
        url_addition = "/map/all/describe"
        r = self._api_call("get", url_addition)
        return r.json()["models"]
    



In [ ]:
et_layer_details <- function(session,cookie){
   # get all layers
   l = get_all_layers(session,cookie)
   m = l$models
   aoi=list()
   # sequence along models
   for (i in seq_along(m)) {
     # parse model
     for (j in seq_along(m[[i]]$layers)) {
       # find layers in model
       aoi[[length(aoi)+1]] <- list(m[[i]]$layers[[j]]$name, m[[i]]$layers[[j]]$id, m[[i]]$id, m[[i]]$layers[[j]]$geometryType) 
     }
   }
   df <- as.data.frame(do.call(rbind, aoi))  
   names(df) <- c('layerName','lid','pid','geometryType')
   # drop default and track layers
   df <- df %>% filter(!pid %in% c('track','default'))
   return(df)
 }

In [ ]:
x = testNewAPi(username, password)
x._get_all_layers()

In [ ]:
# api/map/all/describe
get_all_layers <- function(session=session,
                           URL = "https://focus.sensingclues.org/",
                           cookie=cookie){
  url = paste0(URL,'api/map/all/describe')
  # initial call to get total and number of pages to get
  handle_reset(URL)
  result <- GET(url, set_cookies(focus2 = URLdecode(cookie$value)))
  layers <- content(result)
  return(layers)
}

# get layer details name pid and lid as dataframe
get_layer_details <- function(session,URL = "https://focus.sensingclues.org/",cookie){
  # get all layers
  l = get_all_layers(session,URL,cookie)
  m = l$models
  aoi=list()
  # sequence along models
  for (i in seq_along(m)) {
    # parse model
    for (j in seq_along(m[[i]]$layers)) {
      # find layers in model
      aoi[[length(aoi)+1]] <- list(m[[i]]$layers[[j]]$name, m[[i]]$layers[[j]]$id, m[[i]]$id, m[[i]]$layers[[j]]$geometryType) 
    }
  }
  df <- as.data.frame(do.call(rbind, aoi))  
  names(df) <- c('layerName','lid','pid','geometryType')
  # drop default and track layers
  df <- df %>% filter(!pid %in% c('track','default'))
  return(df)
}